# Specular reflection and transimission

The behavior of light at perfectly smooth surfaces can be characterize by [Fresnel equation and Snell's Law](../fresnel_equation.ipynb), and it's easy to solve analytically.  

Let's define the notions:  
$W_i$ : incoming vector  
$L_i$ : incoming radiance  
$\eta_i$ : IOR of exterior medium  
$W_o$ : outgoing vector  
$L_o$ : outgoing radiance  
$\eta_o$ : IOR of interior medium  
$F_r$ : fresnel  
$f$ : BRDF or BTDF  
$\delta$ : delta distribution


## Specular reflection

In reflection case, the amout of reflected light is $L_i*F_r$, so we have
\begin{align}
L_o(W_o) &= \int_{hemisphere} {f(W_i, W_o)} {L_i(W_i)} {\overline{cos(\theta_i)}} d{W_i} = L_i(W_i) F_r(W_i, \eta_i, \eta_o)
\end{align}

For perfectly smooth surface, there is only one direction that will reflect light, so that BRDF will be delta distribution and integral can be removed. Then we have
\begin{align}
f(W_i, W_o) = \frac {F_r(W_i, \eta_i, \eta_o) \delta(W_i, Reflect(W_o))} {\overline{cos(\theta_i)}}
\end{align}

We also noticed that $\theta_i = \theta_o$ and $\phi_i = \phi_o + \pi$, so BRDF is reciprocity which means $f(W_i, W_o) = f(W_o, W_i)$

## Specular transmission

For transmission, the amout of reflected light is $L_i*(1 - F_r)$, but be carefull with solid angle compression. Here are simple description from pbrt book

> When light refracts into a material with a higher index of refraction than the incident  
medium’s index of refraction, the energy is compressed into a smaller set of angles. This is  
easy to see yourself, for instance, by looking at the sky from underwater in a quiet outdoor  
swimming pool. Because no light can be refracted below the critical angle (∼ 48.6◦ for  
water), the incident hemisphere of light is squeezed into a considerably smaller subset  
of the hemisphere, which covers the remaining set of angles. Radiance along rays that  
do refract must thus increase so that energy is preserved when light passes through the  
interface. More precisely, the incident (Li) and transmitted (Lt) radiance are related by  

We can derive the equation with following steps
\begin{align}
d{\Phi_o} &= {(1 - F_r(W_i, \eta_i, \eta_o))} d{\Phi_i} \\
L_o(W_o) d{W_o} cos(\theta_o) d{A} &= {(1 - F_r(W_i, \eta_i, \eta_o))} L_i(W_i) d{W_i} cos(\theta_i) d{A} \\
L_o(W_o) sin(\theta_o) d{\theta_o} d{\phi_o} cos(\theta_o) d{A} &= {(1 - F_r(W_i, \eta_i, \eta_o))} L_i(W_i) sin(\theta_i) d{\theta_i} d{\phi_i} cos(\theta_i) d{A}
\end{align}

With Snell's law, it can be simplied
\begin{align}
L_o(W_o) = \frac {\eta_o^2} {\eta_i^2} {(1 - F_r(W_i, \eta_i, \eta_o))} L_i(W_i)
\end{align}

Then we have
\begin{align}
f(W_i, W_o) = \frac {\eta_o^2} {\eta_i^2} {(1 - F_r(W_i, \eta_i, \eta_o))} L_i(W_i) \frac {\delta(W_i, Refract(W_o))} {\overline{cos(\theta_i)}}
\end{align}

And we noticed that BTDF doesn't reciprocal due to solid angle compression.

# Microfacet models

There are several awesome papers that give very detailed explanation about microfacet theory, check it if you don't:  
* [Theory for Off-Specular Reflection From Roughened Surfaces](http://www.graphics.cornell.edu/~westin/pubs/TorranceSparrowJOSA1967.pdf)
* [Microfacet Models for Refraction through Rough Surfaces](https://dl.acm.org/citation.cfm?id=2383874)
* [Understanding the Masking-Shadowing Functionin Microfacet-Based BRDFs](http://jcgt.org/published/0003/02/03/paper.pdf)

We can obtain BRDF and BTDF for microfacet models with following setup and derivation.

![](../images/setting_for_the_derivation_of_the_torrance_sparrow_model.png)

### BRDF

Firstly, we need to measure power of energy in microface, then convert to macrosurface.

\begin{align}
d{\Phi_h} &= L_i(W_i) d{W_i} cos(\theta_h) dA(W_h) \\
          &= L_i(W_i) d{W_i} cos(\theta_h) D(W_h) G(W_h) dA
\end{align}

Some part of light is reflected base on fresnel $d{\Phi_o} = F_r d{\Phi_h}$

\begin{align}
d{\Phi_o} &= F_r L_i(W_i) d{W_i} cos(\theta_h) D(W_h) G(W_h) d{W_h} dA \\
          &= L_o(W_o) d{W_o} cos(\theta_o) dA
\end{align}

So

\begin{align}
L_o(W_o) &= \frac {F_r L_i(W_i) d{W_i} cos(\theta_h) D(W_h) G(W_h) d{W_h}}  {d{W_o} cos(\theta_o)} \hspace{1cm} \text{where} \hspace{1cm} d{W_h} = \frac {d{W_o}} {4 cos(\theta_h)} \\
         &= \frac {F_r L_i(W_i) d{W_i} D(W_h) G(W_h)}  {4 cos(\theta_o)}
\end{align}

By the definition of BRDF $$f = \frac {L_o} {d{E_i}} = \frac {L_o} {L_i(W_i) * cos(\theta_i) * d{W_i}}$$

Now we have
\begin{align}
f(W_i, W_o) = \frac {F_r D(W_h) G(W_h)}  {4 cos(\theta_o) cos(\theta_i)}
\end{align}

Still this BRDF is reciprocity.

### BTDF

Derivation of BTDF is almost the same as BRDF except that
$$d{W_h} = d{W_o} \frac {\eta_o^2 |(W_o \cdot W_h)|} {(\eta_i (W_i \cdot W_h) + \eta_o (W_o \cdot W_h))^2}$$

So we have BTDF
\begin{align}
f(W_i, W_o) = {\frac {(1 - F_r) D(W_h) G(W_h)} {cos(\theta_o) cos(\theta_i)}} {\frac {\eta_o^2 |W_o \cdot W_h| |W_i \cdot W_h |} {(\eta_i (W_i \cdot W_h) + \eta_o (W_o \cdot W_h))^2}}
\end{align}

It's important to notice that BTDF is not reciprocity, and we have $$\frac {f(W_i, W_o)} {\eta_o^2} = \frac {f(W_o, W_i)} {\eta_i^2}$$